## Counterfactual Instances
https://docs.seldon.io/projects/alibi/en/latest/methods/CF.html#Numerical-Gradients

In [1]:
!pip install alibi

  Using cached numpy-1.19.5-cp38-cp38-manylinux2010_x86_64.whl (14.9 MB)
  Using cached six-1.15.0-py2.py3-none-any.whl (10 kB)
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.3
    Uninstalling numpy-1.20.3:
      Successfully uninstalled numpy-1.20.3


In [2]:
import tensorflow as tf
tf.get_logger().setLevel(40) # suppress deprecation messages
tf.compat.v1.disable_v2_behavior() # disable TF2 behaviour as alibi code still relies on TF1 constructs
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import to_categorical
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.datasets import load_boston
from alibi.explainers import CounterFactualProto, CounterFactual
import pandas as pd
from learning import *
from time import time

print('TF version: ', tf.__version__)
print('Eager execution enabled: ', tf.executing_eagerly()) # False
seed = 123
tf.random.set_seed(seed)
np.random.seed(seed)

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

TF version:  2.4.1
Eager execution enabled:  False


# Install alibi

In [3]:
## Removing str encoding error.
!python3 -m pip install 'h5py==2.10.0' --force-reinstall

  Using cached h5py-2.10.0-cp38-cp38-manylinux1_x86_64.whl (2.9 MB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Using cached numpy-1.20.3-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.4 MB)
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: h5py
    Found existing installation: h5py 2.10.0
    Uninstalling h5py-2.10.0:
      Successfully uninstalled h5py-2.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.4.1 requires numpy~=1.19.2, but you have numpy 1.20.3 which is incompatible.
tensorflow 2.4.1 requires six~=1.15.0, but you have six 1.16.0 which is inco

# Load Diabetes dataset

In [4]:
# Giving current root path
PATH = "./"

# name of dataset
DATASET_NAME = "diabetes.csv"

# variable containing the class labels in this case the dataset contains:
# 0 - if not diabetes
# 1 - if diabetes
class_var = "Outcome"

# load dataset
dataset_path = PATH + "datasets/" + DATASET_NAME
data = pd.read_csv( dataset_path )

# features
feature_names = data.drop([class_var], axis=1).columns.to_list()

# balance dataset
sampled_data = data.sample(frac=1)
sampled_data = sampled_data[ sampled_data["Outcome"] == 0]

no_data = sampled_data.sample(frac=1)[0:268]
yes_data = data[ data["Outcome"] == 1]

balanced_data = [no_data,yes_data]
balanced_data = pd.concat(balanced_data)

# apply one hot encoder to data
# standardize the input between 0 and 1
X, Y, encoder, scaler = encode_data( balanced_data, class_var)

n_features = X.shape[1]
n_classes = len(data[class_var].unique())

# load existing training data
print("Loading training data...")
X_train, Y_train, X_test, Y_test, X_validation, Y_validation= load_training_data( dataset_path )

print("====================Features====================")
print(feature_names)
print("================================================")

Loading training data...
====================Features====================
['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']


# Load trained model

In [5]:
# the best performing model was obtained with 5 hidden layers with 12 neurons each
model_name = "model_h5_N12"

# specify paths where the blackbox model was saved
path_serialisation_model = PATH + "training/" + DATASET_NAME.replace(".csv", "") + "/model/" 
path_serialisation_histr = PATH + "training/" + DATASET_NAME.replace(".csv", "") + "/history/" 

# load model and model performance history
print("Loading Blackbox model...")
model_history = load_model_history( model_name, path_serialisation_histr )
model = load_model( model_name, path_serialisation_model )

# check modelxw
model.summary()

Loading Blackbox model...
Loaded model from disk
Model: "model_h5_N12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_293 (Dense)            (None, 12)                108       
_________________________________________________________________
dense_294 (Dense)            (None, 12)                156       
_________________________________________________________________
dense_295 (Dense)            (None, 12)                156       
_________________________________________________________________
dense_296 (Dense)            (None, 12)                156       
_________________________________________________________________
dense_297 (Dense)            (None, 12)                156       
_________________________________________________________________
dense_298 (Dense)            (None, 12)                156       
_________________________________________________________________
dense

In [6]:
## Dentermine the feature range by training set.
diabetes_feature_range = (X_train.min(axis=0), X_train.max(axis=0))

In [8]:
data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [9]:
## Get an example instance from test set.
example_idx = 0 ## Could be change!!
example_data = np.expand_dims(X_test[example_idx], axis=0)

In [10]:
cf_b = CounterFactual(model, example_data.shape, distance_fn='l1', target_proba=1.0,
                    target_class='other', max_iter=1000, early_stop=50, lam_init=1e-1,
                    max_lam_steps=10, tol=0.05, learning_rate_init=0.1,
                    feature_range=diabetes_feature_range, eps=0.01, init='identity',
                    decay=True, write_dir=None, debug=False)

`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.


In [11]:
start_time = time()
explanation_b = cf_b.explain(example_data)
print('Explanation took {:.3f} sec'.format(time() - start_time))

Explanation took 10.021 sec


In [12]:
def log_cf_found(data, scaler, explanation):
    '''
    Function for generating measuring the difference btw original instance and counterfactual.
    '''
    print("================================================")    
    print(f'Original prediction: {explanation.orig_class}')
    print('Counterfactual prediction: {}'.format(explanation.cf['class']))
    print("================================================")    
    pred_class = explanation.cf['class']
    proba = explanation.cf['proba'][0][pred_class]
    print(f'Counterfactual prediction: {pred_class} with probability {proba}')
    print("================================================")    
    orig =  scaler.inverse_transform(data)
    counterfactual = scaler.inverse_transform(explanation.cf['X'])
    print(counterfactual)
    delta = counterfactual - orig
    for i, f in enumerate(feature_names):
        if np.abs(delta[0][i]) > 1e-4:
            print('{}: {}'.format(f, delta[0][i]))
    print("================================================")    
    

In [13]:
counterfactual = scaler.inverse_transform(example_data)
counterfactual

array([[1.00000000e+00, 1.89000000e+02, 5.60655738e+01, 2.30000000e+01,
        8.46000000e+02, 3.01000000e+01, 3.91938330e-01, 5.90000000e+01]])

In [14]:
log_cf_found(example_data, scaler, explanation_b)

Original prediction: 1
Counterfactual prediction: 0
Counterfactual prediction: 0 with probability 0.9502091407775879
[[  0.9848623 188.46289     0.          0.        179.63701    30.110416
    0.3963714  72.       ]]
Pregnancies: -0.015137672424316628
Glucose: -0.5371093749999432
BloodPressure: -56.06557377049181
SkinThickness: -23.0
Insulin: -666.3629913330078
BMI: 0.010416412353517757
DiabetesPedigreeFunction: 0.004433064631374273
Age: 13.000000000000007


In [15]:
example_data

array([[0.05882353, 0.94974874, 0.49180328, 0.23232323, 1.        ,
        0.4485842 , 0.13404711, 0.74509804]])